Modelo de aprendizaje automático:

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. Deben crear al menos uno de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:

Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.
Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

# Modelamiento

In [507]:
# Importar librerias

import pandas as pd

### 1- Obtener datos provenientes del EDA

In [508]:
# Lectura de CSV en un df
ruta = "eda_modelo.csv"
df_modelo = pd.read_csv(ruta)

### 2- Segmentar y agrupar

In [509]:
# Revisar la cantidad de tipos de datos y valores nulos por cada columna de un dataframe

def count_column_types(df):
    '''
    Obtener los tipos de datos unicos y sus respectivos recuentos para cada columna.
    Funcion que recibe como parámetro un dataframe
    '''
    
    result = df.apply(lambda col: col.apply(type) .value_counts())          # Aplicar la función a todas las columnas del DataFrame
    result = result.T.reset_index()                                         # Transponer el resultado y restablecer el índice
    result. columns = ["Column"] + result.columns[1:].tolist()              # Renombrar las columnas

    serie_nulos = df.isnull().sum()                                         # Hallamos los valores nulos y se almacenan en una serie
    serie_nulos = serie_nulos.reset_index()                                 # Convierte el índice en una columna
    serie_nulos.columns = ['Column', 'Nulos']                               # Asigna un nombre a la columna del índice si es necesario

    df_1 = result
    df_2 = serie_nulos
    columna_union = "Column"
    df_merge = df_1.merge(df_2, on=columna_union, how='left')               # Realizar un left join entre df_userdata y df_games

    df_merge["Nulos %"] = round(df_merge["Nulos"] / df.shape[0] * 100, 2)   # Porcentaje total de registros nulos

    print("\nfilas completamente nulas: ", df.isna().all(axis=1).sum())     # Filas que se encuentran totalmente en nulo

    return df_merge

2-1- Descartar variables redundantes

In [510]:
df_modelo.head(5)

,item_id,app_name,genres,tags
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Strategy', 'Action', 'Indie', 'Casual', 'Sim..."
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']","['Free to Play', 'Strategy', 'Indie', 'RPG', '..."
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Simulation', 'Sports', 'Casu..."
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']","['Action', 'Adventure', 'Casual']"
4,773570,Log Challenge,NaN,"['Action', 'Indie', 'Casual', 'Sports']"


In [511]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32132 entries, 0 to 32131
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   item_id   32132 non-null  int64 
 1   app_name  32131 non-null  object
 2   genres    28850 non-null  object
 3   tags      31970 non-null  object
dtypes: int64(1), object(3)
memory usage: 1004.3+ KB


In [512]:
count_column_types(df_modelo)


filas completamente nulas:  0


,Column,<class 'int'>,<class 'str'>,<class 'float'>,Nulos,Nulos %
0,item_id,32132.0,NaN,NaN,0,0.00
1,app_name,NaN,32131.0,1.0,1,0.00
2,genres,NaN,28850.0,3282.0,3282,10.21
3,tags,NaN,31970.0,162.0,162,0.50


In [513]:
# Elimina registros en los que ambas columnas "genres" y "tags" estén en nulo
df_modelo = df_modelo.dropna(subset=["genres", "tags"], how='all')

In [514]:
count_column_types(df_modelo)


filas completamente nulas:  0


,Column,<class 'int'>,<class 'str'>,<class 'float'>,Nulos,Nulos %
0,item_id,31994.0,NaN,NaN,0,0.00
1,app_name,NaN,31993.0,1.0,1,0.00
2,genres,NaN,28850.0,3144.0,3144,9.83
3,tags,NaN,31970.0,24.0,24,0.08


In [515]:
# Ajustar el tipo de dato de cada columna
df_modelo["genres"] = df_modelo["genres"].astype(str)
df_modelo["tags"] = df_modelo["tags"].astype(str)

# Crear una nueva columna llamada 'combined' que contiene listas de géneros y etiquetas
df_modelo['combined'] = df_modelo['genres'] + ',' + df_modelo['tags']

# eliminar columnas irrelevantes
df_modelo = df_modelo[["item_id", "app_name", "combined"]]

df_modelo.head(10)

,item_id,app_name,combined
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy'],[..."
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual'],['Action', '..."
4,773570,Log Challenge,"nan,['Action', 'Indie', 'Casual', 'Sports']"
5,772540,Battle Royale Trainer,"['Action', 'Adventure', 'Simulation'],['Action..."
6,774276,SNOW - All Access Basic Pass,"['Free to Play', 'Indie', 'Simulation', 'Sport..."
7,774277,SNOW - All Access Pro Pass,"['Free to Play', 'Indie', 'Simulation', 'Sport..."
8,774278,SNOW - All Access Legend Pass,"['Free to Play', 'Indie', 'Simulation', 'Sport..."
9,768800,Race,"['Casual', 'Indie', 'Racing', 'Simulation'],['..."


In [516]:
count_column_types(df_modelo)


filas completamente nulas:  0


,Column,<class 'int'>,<class 'str'>,<class 'float'>,Nulos,Nulos %
0,item_id,31994.0,NaN,NaN,0,0.0
1,app_name,NaN,31993.0,1.0,1,0.0
2,combined,NaN,31994.0,NaN,0,0.0


In [517]:
'''
# Funcion para reemplazar caracteres
def reemplazar_str(cadena):
    cadena = cadena.lower()
    valores_a_reemplazar = ["[", "]", "'", " "]
    for character in valores_a_reemplazar:
        cadena = cadena.replace(character,'')
    return cadena

# Funcion eliminar duplicados de una lista
def remover_duplicados(lista):
    mi_lista = lista
    mi_lista_sin_duplicados = []
    for elemento in mi_lista:
        if (elemento not in mi_lista_sin_duplicados) and (elemento != "nan"):
            mi_lista_sin_duplicados.append(elemento)
    return mi_lista_sin_duplicados

# Funcion para reemplazar caracteres
def reemplazar_list(lista):
    nueva_lista = []
    for valor in lista:
        nuevo_valor = reemplazar_str(valor)
        nueva_lista.append(nuevo_valor)
    nueva_lista = remover_duplicados(nueva_lista)
    return nueva_lista



# Eliminar caracteres llaves para integrar los str y convertir a listas
df_modelo["editado"] = df_modelo["combined"].apply(reemplazar)

# Convertir las cadenas combinadas en listas de Python
df_modelo['details'] = df_modelo['editado'].apply(lambda x: x.split(','))

# Limpiar caracteres en detail_list y filtar valores unicos
df_modelo['details'] = df_modelo['details'].apply(reemplazar_list)

# reducir dataframe
df_modelo = df_modelo[["item_id", "app_name", "details"]]

df_modelo.head(10)
'''

'\n# Funcion para reemplazar caracteres\ndef reemplazar_str(cadena):\n    cadena = cadena.lower()\n    valores_a_reemplazar = ["[", "]", "\'", " "]\n    for character in valores_a_reemplazar:\n        cadena = cadena.replace(character,\'\')\n    return cadena\n\n# Funcion eliminar duplicados de una lista\ndef remover_duplicados(lista):\n    mi_lista = lista\n    mi_lista_sin_duplicados = []\n    for elemento in mi_lista:\n        if (elemento not in mi_lista_sin_duplicados) and (elemento != "nan"):\n            mi_lista_sin_duplicados.append(elemento)\n    return mi_lista_sin_duplicados\n\n# Funcion para reemplazar caracteres\ndef reemplazar_list(lista):\n    nueva_lista = []\n    for valor in lista:\n        nuevo_valor = reemplazar_str(valor)\n        nueva_lista.append(nuevo_valor)\n    nueva_lista = remover_duplicados(nueva_lista)\n    return nueva_lista\n\n\n\n# Eliminar caracteres llaves para integrar los str y convertir a listas\ndf_modelo["editado"] = df_modelo["combined"].appl

In [518]:
# Usar str.lower() para convertir la columna a minúsculas
df_modelo['combined'] = df_modelo['combined'].str.lower()

# limpiar cadena para get_dummies
reemplazar = ["[", "]", "'", " ", '"']

for var in reemplazar:
    df_modelo['combined'] = df_modelo['combined'].str.replace(var, '').astype(str)

for var in reemplazar:
    df_modelo['combined'] = df_modelo['combined'].str.replace(";", ',').astype(str)

# Función para eliminar palabras repetidas en un registro
def eliminar_palabras_repetidas(cadena):
    cadena = cadena.split(",")  # Dividir la cadena en palabras
    palabras_unicas = list(set(cadena))  # Convertir a conjunto para eliminar duplicados y luego volver a lista
    texto_sin_repetidos = ', '.join(palabras_unicas) # Volver a unir las palabras en una cadena separada por comas
    return texto_sin_repetidos

# Aplicar la función a la columna 'combined'
df_modelo['combined'] = df_modelo['combined'].apply(eliminar_palabras_repetidas)

df_modelo

,item_id,app_name,combined
0,761140,Lost Summoner Kitty,"action, simulation, casual, strategy, indie"
1,643980,Ironbound,"difficult, cardgame, turn-based, fantasy, pvp,..."
2,670290,Real Pool 3D - Poolians,"simulation, sports, casual, multiplayer, indie..."
3,767400,弹炸人2222,"casual, action, adventure"
4,773570,Log Challenge,"action, sports, casual, indie, nan"
...,...,...,...
32127,773640,Colony On Mars,"casual, strategy, simulation, indie"
32128,733530,LOGistICAL: South Africa,"casual, strategy, indie"
32129,610660,Russian Roads,"simulation, racing, indie"
32130,658870,EXIT 2 - Directions,"puzzle, relaxing, casual, indie, atmospheric, ..."


In [519]:
# Crear un DataFrame con las columnas de géneros
df_details = df_modelo['combined'].str.get_dummies(', ')
df_details = df_details.drop(columns="nan")
df_details

,1980s,1990s,2.5d,2d,2dfighter,360video,3dplatformer,3dvision,4playerlocal,4x,...,wargame,warhammer40k,webpublishing,werewolves,western,wordgame,worldwari,worldwarii,wrestling,zombies
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [520]:
# Conteo de la cantidad de registros involucrados para cada columna y su porcentaje de participación en el dataframe

def contar_registros_x_columna(df):
    mi_dict = {"detail": [], "sum": [], "percent": []}

    for column in df.columns:
        mi_dict["detail"].append(column)
        mi_dict["sum"].append(df[column].sum())
        porcentaje = round((df[column].sum() / len(df)) * 100, 2)
        mi_dict["percent"].append(porcentaje)
        df_count = pd.DataFrame(mi_dict).sort_values(by='sum', ascending=False)

    return df_count

In [521]:
# revisar la participacion de cada columna en los datos
df_details_count = contar_registros_x_columna(df_details)
df_details_count.reset_index(drop=True)

,detail,sum,percent
0,indie,17642,55.14
1,action,13016,40.68
2,adventure,9840,30.76
3,casual,9767,30.53
4,simulation,7749,24.22
...,...,...,...
339,intentionallyawkwardcontrols,6,0.02
340,voicecontrol,5,0.02
341,cycling,5,0.02
342,faith,4,0.01


In [522]:
# funcion para eliminar columnas
def eliminar_columnas(df, lista_columnas):
    try:
        df.drop(columns=lista_columnas, inplace=True)
    except:
        None
    return 

In [523]:
# eliminar las columnas cuyo porcentaje de participacion sea menor al establecido
porcentaje_limite = 5.0
columnas_eliminar = df_details_count["detail"][df_details_count['percent'] < porcentaje_limite]
eliminar_columnas(df_details, columnas_eliminar)

# revisar la cantidad de columnas resultantes
df_details_count = contar_registros_x_columna(df_details)

df_details_count.reset_index(drop=True)

,detail,sum,percent
0,indie,17642,55.14
1,action,13016,40.68
2,adventure,9840,30.76
3,casual,9767,30.53
4,simulation,7749,24.22
5,strategy,7576,23.68
6,rpg,5929,18.53
7,singleplayer,4344,13.58
8,freetoplay,2413,7.54
9,multiplayer,2383,7.45


In [524]:
# Concatenar el nuevo DataFrame con el original
df_modelo = pd.concat([df_modelo, df_details], axis=1)
df_modelo

,item_id,app_name,combined,2d,action,adventure,atmospheric,casual,earlyaccess,freetoplay,greatsoundtrack,indie,multiplayer,puzzle,rpg,simulation,singleplayer,sports,strategy,vr
0,761140,Lost Summoner Kitty,"action, simulation, casual, strategy, indie",0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
1,643980,Ironbound,"difficult, cardgame, turn-based, fantasy, pvp,...",1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0
2,670290,Real Pool 3D - Poolians,"simulation, sports, casual, multiplayer, indie...",0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,0,0
3,767400,弹炸人2222,"casual, action, adventure",0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,773570,Log Challenge,"action, sports, casual, indie, nan",0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32127,773640,Colony On Mars,"casual, strategy, simulation, indie",0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
32128,733530,LOGistICAL: South Africa,"casual, strategy, indie",0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
32129,610660,Russian Roads,"simulation, racing, indie",0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
32130,658870,EXIT 2 - Directions,"puzzle, relaxing, casual, indie, atmospheric, ...",0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0


In [525]:
df_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31994 entries, 0 to 32131
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item_id          31994 non-null  int64 
 1   app_name         31993 non-null  object
 2   combined         31994 non-null  object
 3   2d               31994 non-null  int64 
 4   action           31994 non-null  int64 
 5   adventure        31994 non-null  int64 
 6   atmospheric      31994 non-null  int64 
 7   casual           31994 non-null  int64 
 8   earlyaccess      31994 non-null  int64 
 9   freetoplay       31994 non-null  int64 
 10  greatsoundtrack  31994 non-null  int64 
 11  indie            31994 non-null  int64 
 12  multiplayer      31994 non-null  int64 
 13  puzzle           31994 non-null  int64 
 14  rpg              31994 non-null  int64 
 15  simulation       31994 non-null  int64 
 16  singleplayer     31994 non-null  int64 
 17  sports           31994 non-null  int

## Construcción del modelo

In [540]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances

In [536]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [528]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = df_modelo.drop(['item_id', 'app_name', "combined"], axis=1)  # Asegúrate de incluir todas las características relevantes
y = df_modelo['item_id']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [535]:
# Calcular la cantidad de elementos a evaluar
n_items = df_modelo["item_id"].unique().shape[0]
n_items

31994

In [542]:
data_matrix = np.zeros((n_items, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

MemoryError: Unable to allocate 7.63 GiB for an array with shape (31994, 31994) and data type float64

In [ ]:
item_similariry = pairwise_distances(data_matrix, metric = cosine)
item_similariry

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#                         

In [529]:
# Definir los hiperparámetros y sus valores posibles
param_grid = {
    'n_estimators': [50, 100, 200],
    'estimator__max_depth': [5, 10, 15],
    'max_samples': [0.7, 0.8, 0.9],
    'max_features': [0.7, 0.8, 0.9]
}

In [532]:
# Crear el modelo de Bagging
bagging_model = BaggingRegressor(estimator=DecisionTreeRegressor(random_state=42),
                                  random_state=42)

In [533]:
# Realizar la búsqueda de cuadrícula
grid_search = GridSearchCV(bagging_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

c:\Users\Ivan Rojas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
c:\Users\Ivan Rojas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
c:\Users\Ivan Rojas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:720: FutureWarning: Parameter 'base_estimator' of BaggingRegressor is deprecated in favor of 'estimator'. See BaggingRegressor's docstring for more details.
  estimator = estimator.set_params(**cloned_parameters)
c:\Users\I

Mejores hiperparámetros encontrados:
{'base_estimator__max_depth': 15, 'max_features': 0.9, 'max_samples': 0.7, 'n_estimators': 200}
